# 🧠 LLM 回答評估：F1 / Semantic / BLEU / ROUGE-L

In [16]:
import warnings
warnings.filterwarnings('ignore')

!pip install -U nltk rouge-score sentence-transformers bert-score

In [5]:
import pandas as pd
import nltk
from nltk import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util
from bert_score import score

In [28]:
# 下載 nltk 資源
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [12]:
# 載入語意模型
semantic_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
# === F1 Score function ===
def compute_f1(pred, truth):
    pred_tokens = word_tokenize(pred.lower())
    truth_tokens = word_tokenize(truth.lower())
    common = set(pred_tokens) & set(truth_tokens)
    if len(common) == 0:
        return 0.0
    precision = len(common) / len(pred_tokens)
    recall = len(common) / len(truth_tokens)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

In [14]:
# === Semantic Similarity ===
def semantic_score(pred, truth):
    emb = semantic_model.encode([pred, truth], convert_to_tensor=True)
    return float(util.pytorch_cos_sim(emb[0], emb[1]))

In [15]:
# === BLEU ===
def bleu_score(pred, truth):
    pred_tokens = word_tokenize(pred.lower())
    truth_tokens = [word_tokenize(truth.lower())]
    return sentence_bleu(truth_tokens, pred_tokens, smoothing_function=SmoothingFunction().method1)


In [6]:
# === ROUGE-L ===
def rouge_l(pred, truth):
    return scorer.score(truth, pred)['rougeL'].fmeasure

# === BERTScore ===
def compute_bertscore(pred, truth):
    # BERTScore typically expects lists of strings
    # It returns P, R, F1. We often use F1.
    # Using lang="en" as the pre-trained SentenceTransformer model 'paraphrase-MiniLM-L6-v2' is English-based.
    # If your text is primarily Chinese, consider changing lang="zh".
    P, R, F1 = score([pred], [truth], lang="en", verbose=False)
    return F1.item() # Get the scalar value

In [7]:
# === Exact Match ===
def exact_match(pred, truth):
    """
    Calculates the Exact Match score between two strings.

    Args:
        pred: The predicted string.
        truth: The ground truth string.

    Returns:
        1.0 if the strings are exactly equal, 0.0 otherwise.
    """
    return 1.0 if pred == truth else 0.0

# Initialize rouge_scorer here after imports
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

In [17]:
# === 載入CSV ===
file_path = '/content/F1_turtler1.csv'  # <- 改成你的檔案名稱
# Try reading with 'big5' encoding
try:
    df = pd.read_csv(file_path, encoding='big5')
except UnicodeDecodeError:
    # If 'big5' fails, try 'cp950' which is also common
    try:
        df = pd.read_csv(file_path, encoding='cp950')
    except Exception as e:
        print(f"Error reading CSV with both 'big5' and 'cp950' encodings: {e}")
        # Handle other potential errors during file reading
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")

In [29]:
# === 計算評估分數 ===
# 先將 'output' 和 'instruction' 欄位明確轉換為字串類型，並處理 NaN 值
df['output'] = df['output'].astype(str).fillna('')
df['instruction'] = df['instruction'].astype(str).fillna('')

df['f1'] = df.apply(lambda row: compute_f1(row['output'], row['instruction']), axis=1)
df['semantic'] = df.apply(lambda row: semantic_score(row['output'], row['instruction']), axis=1)
df['rougeL'] = df.apply(lambda row: rouge_l(row['output'], row['instruction']), axis=1)
df['bertscore'] = df.apply(lambda row: compute_bertscore(row['output'], row['instruction']), axis=1)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho

In [30]:
# === 顯示平均值 ===
print(f"平均 F1-score：{df['f1'].mean():.4f}")
print(f"平均 Semantic similarity：{df['semantic'].mean():.4f}")
print(f"平均 ROUGE-L：{df['rougeL'].mean():.4f}")
print(f"平均 BERTScore：{df['bertscore'].mean():.4f}")

平均 F1-score：0.3750
平均 Semantic similarity：0.7143
平均 ROUGE-L：0.3750
平均 BERTScore：0.8752


In [31]:
# 可選：匯出加上分數的CSV
df.to_csv('/content/eval_results.csv', index=False)
print("已儲存評估結果為 eval_results.csv")

已儲存評估結果為 eval_results.csv


In [24]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True